# The purpose of this notebook is to condense and simplify the running of FoldSeek, and useful downstream processes into a single Jupyter Notebook. The objective is that a user can define several parameters and directories and then click 'Run all' and leave the notebook to finish for however long it takes.

# This is meant to serve as an example so PLEASE DO NOT MODIFY, to use this notebook, please setup a new directory and copy in this notebook (you'll have to change some direcotry names for this), or run the accompanying script via the command line - this will take as inputs the paths to various directories it needs via the command line

In [104]:
import os
import numpy as np
import pandas as pd
import time
import glob
%run functions.py

### A few things to note:

1. Camel Case is used AT ALL TIMES for variables, Snake case is used for functions - this way if there are any name clashes functions and variables can be distinguished by the user

## Parameters

### Parameters and directories for FoldSeek
You need to create the search directory where everything will go and fill it witha  sub-directory of query proteins, everything else should be done for you. Check everything makes sense at this stage!

In [119]:
CurrentSearchDirectory = 'FoldSeekExamples' # The directory which houses and will house all sub-directories e.g. CancerCureFoldSeekSearch
QueryPath = f'{CurrentSearchDirectory}/ProteinQueries'
assert os.path.exists(QueryPath), f"Directory does not exist: {QueryPath}" # Assert that the folder with protein queries already exists
QueryProteins = list_files_in_directory(QueryPath)
assert (len(QueryProteins)>0), f'No proteins in directory: {QueryPath}'
QueryProteins

['FoldSeekExamples/ProteinQueries/1rei.pdb',
 'FoldSeekExamples/ProteinQueries/1mbn.pdb']

In [122]:
DatabasePath = 'databases/' # The databases that FoldSeek will look through are in this directory - there are 5
Databases = list_folders_in_directory(DatabasePath)
Databases = [f"{element.rsplit('/', 1)[0]}/{element.rsplit('/', 1)[1]}/{element.rsplit('/', 1)[1].lower()}" if '/' in element else element for element in Databases]
assert (len(Databases)==5), f'Five databases should be present in the database directory!'
Databases

['databases/UP/up',
 'databases/ESMA/esma',
 'databases/AFDB/afdb',
 'databases/PDB/pdb',
 'databases/SP/sp']

In [4]:
# The below variables will tell FoldSeek what to output, and the corresponding column names they will be stored in in the final CSV
# I highly reccomend looking at the Github for FoldSeek and MMSeq2 which detail what these variables are adn how they're computed
# Once this is studied, you will notice tCa and qCa (co-ordinates of query and target alpha carbons) are missing - they were 
# creating issues with the tab-separated readout, this may be corrected in future updates so check if this is still the case if you think
# these variables will be useful to you.

FormatOutput = ("--format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,tseq,evalue,bits,cigar,"
                "alntmscore,qtmscore,ttmscore,rmsd,u,t,lddt,lddtfull,prob,taxid,taxname,taxlineage") 

Columns = ['query','target','fident','alnlen','mismatch','gapopen','qstart','qend','tstart','tend','tseq','evalue','bits','cigar',
           'alntmscore','qtmscore','ttmscore','rmsd','u','t','lddt','lddtfull','prob','taxid','taxname','taxlineage','database']

# The ESMA database has no taxonomic data, and so requires its own handling
FormatOutputESMA = ("--format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,tseq,evalue,bits,cigar,"
                "alntmscore,qtmscore,ttmscore,rmsd,u,t,lddt,lddtfull,prob") 


In [5]:
CSVFileDirectory = f"{CurrentSearchDirectory}/FoldSeekExtractionOutputCSVs" # Several CSV files will be generated and placed here
check_existence(CSVFileDirectory)
CreateHTMLs = True # Will we generate HTMLs? - expect a doubling of runtime if you do
if CreateHTMLs:
    HTMLFileDirectory = f"{CurrentSearchDirectory}/FoldSeekExtractionOutputHTMLs" # If we are making HTMLs we'll put them here
    check_existence(HTMLFileDirectory)
AlignmentAndTempFileDirectory = f"{CurrentSearchDirectory}/FoldSeekExtractionOutputAlntmps"
check_existence(AlignmentAndTempFileDirectory)

Directory 'FoldSeekExamples/FoldSeekExtractionOutputCSVs' was created.
Directory 'FoldSeekExamples/FoldSeekExtractionOutputHTMLs' was created.
Directory 'FoldSeekExamples/FoldSeekExtractionOutputAlntmps' was created.


### Parameters and directories for BLASTs
These parameters are almost entirely user-dependent so all of this is up to you! Put your databases in the BLASTDatabases folder, reading documentation on the BLAST functions will probably be very useful. Something to bear in mind is that you can create a BLASTable database from a fasta file with all the sequences you want, or download a database with all the needed files ready-made from NCBI etc. Examples of both are shown here

In [82]:
BLASTOutputDirectory = f'{CurrentSearchDirectory}/BLAST_XMLs' # BLASTs will be saved and parsed as .xml files, please read documentation if not familiar
check_existence(BLASTOutputDirectory)
BLASTList = ['BPPRC','AllergenOnline','Patent']# The list of BLASTs that we will be doing, in the order we will do them
BLASTDirectories = [f'BLASTDatabases/{BLAST}_BLAST_DB' for BLAST in BLASTList]

Directory 'FoldSeekExamples/BLAST_XMLs' was created.


In [125]:
for directory in BLASTDirectories: # make sure they exist!
    check_and_assert_existence(directory)

Directory 'BLASTDatabases/BPPRC_BLAST_DB' already exists.
Directory 'BLASTDatabases/AllergenOnline_BLAST_DB' already exists.
Directory 'BLASTDatabases/Patent_BLAST_DB' already exists.


In [72]:
RebuildDBs = True # If this is true, an if statement will be entered where the BLASTable database is built up from a single fasta file
DBsToRebuild = ['BPPRC','AllergenOnline'] # What DBs need to be rebuilt in such a way - in the example these two, 
                                          #the patent is from NCBI so will already contain everything necessary (and doesn't have a fasta!)
if RebuildDBs:
    for DB in DBsToRebuild:
        PathToDB = f'BLASTDatabases/{DB}_BLAST_DB'
        DBFASTA = glob.glob(f'{PathToDB}/*.fa')[0] # finds the fasta in the directory - obviously make sure its there!!!
        build_blast_database(DBFASTA, dbtype='prot') # Builds out the necessary files for BLASTing

### Parameters for filtering based on structural (FoldSeek) or sequential (BLAST) alignments
The world is your oyster, here are some examples that will probably hold true for most runs, but you could add a filter based on if the taxname is Equus unicornus, or if a certain subsequence appears in the target sequence, or if the BLAST description says 'Cures all cancers'

In [123]:
TMScoreThreshold = 0.4 # NOTE: this will be applied to ttmscore AND qtmscore, if you don't know what these are, please read the documentation!
BLASTBitscoreThresholds = [100,100,50] # Only those sequences whose *BEST* BLAST is a worse match than these bitscores go through
                                       # In this example, 100 for BPPRC,100 for AllergenOnline, 50 for Patent
                                       # To get a feel of bitscore read documentation, and visualise yourself!
MustStartWithMethionine=True    # If you want to clone it, it needs to have a start codon...
RemoveDuplicates = True # Get rid of duplicate proteins so we have an idea of what we really want to keep

# Code
## Obtaining variables from FoldSeek
### I reccomend looking at the FoldSeek Github repo to understand the command line commands being built here

In [12]:
FirstFoldSeekdf = pd.DataFrame(columns=Columns) # This is the first major dataframe, it will be the largest, and saved at the end of the cell
# Loop through all the Query proteins
for QueryProtein in QueryProteins:
    PDBidentifier = QueryProtein.split('/')[-1].split('.')[-2] # This should hold true for any path to any .pdb file, this variable is used for naming of files

    for Database in Databases:     # Loop through all the databases
        
        #First make the HTMLS (This doesn't depend on if the database is ESMA or not)
        if CreateHTMLs:
                os.system(f"foldseek easy-search {QueryProtein} {Database} {HTMLFileDirectory}/{PDBidentifier}_{Database.split('/')[-1]}.html {AlignmentAndTempFileDirectory}/tmp --format-mode 3")
                os.system(f"rm -r {AlignmentAndTempFileDirectory}/tmp") # remove the temporary file (for storage and sanity)
            
        # Generate the alignment file - with a different output format if the database is ESMA    
        if Database.split('/')[-1] == 'esma':
            os.system(f"foldseek easy-search {QueryProtein} {Database} {AlignmentAndTempFileDirectory}/aln_{PDBidentifier}_{Database.split('/')[-1]} {AlignmentAndTempFileDirectory}/tmp {FormatOutputESMA}")
            df = pd.read_csv(f"{AlignmentAndTempFileDirectory}/aln_{PDBidentifier}_{Database.split('/')[-1]}",sep='\t', header=None) # Read the current alignment file (it is a tab separated file)
            df.columns = Columns[:-4]
        else:
            os.system(f"foldseek easy-search {QueryProtein} {Database} {AlignmentAndTempFileDirectory}/aln_{PDBidentifier}_{Database.split('/')[-1]} {AlignmentAndTempFileDirectory}/tmp {FormatOutput}")
            df = pd.read_csv(f"{AlignmentAndTempFileDirectory}/aln_{PDBidentifier}_{Database.split('/')[-1]}",sep='\t', header=None) # Read the current alignment file (it is a tab separated file)
            df.columns = Columns[:-1]
        
        df['database'] = Database # Add in what database the current alignments are from
        df = df.reindex(columns=Columns) # Just to make sure everything is as it should be - the ESMA df should have three NaN columns
        
        os.system(f"rm -r {AlignmentAndTempFileDirectory}/tmp") # remove the temporary directory (for storage and sanity)
        FirstFoldSeekdf = pd.concat([df,FirstFoldSeekdf],axis=0) # Add the current alignment to the main dataframe

FirstFoldSeekdf.to_csv(f'{CSVFileDirectory}/FirstFoldSeekdf.csv',index=False) # Save this dataframe to a csv for future manipulation

FoldSeekExamples/FoldSeekExtractionOutputHTMLs/1rei_up.html exists and will be overwritten
easy-search FoldSeekExamples/ExampleQueries/1rei.pdb databases/UP/up FoldSeekExamples/FoldSeekExtractionOutputHTMLs/1rei_up.html FoldSeekExamples/FoldSeekExtractionOutputAlntmps/tmp --format-mode 3 

MMseqs Version:              	5433d6db83522482410b30b180001eeb269f7505
Seq. id. threshold           	0
Coverage threshold           	0
Coverage mode                	0
Max reject                   	2147483647
Max accept                   	2147483647
Add backtrace                	false
TMscore threshold            	0
TMalign hit order            	0
TMalign fast                 	1
Preload mode                 	0
Threads                      	72
Verbosity                    	3
LDDT threshold               	0
Sort by structure bit score  	1
Alignment type               	2
Substitution matrix          	aa:3di.out,nucl:3di.out
Alignment mode               	3
Alignment mode               	0
E-value threshold

### This may be a good point to make your own notebook and investigate the data held in the newly created csv yourself

In [60]:
FirstFoldSeekdf = pd.read_csv(f'{CSVFileDirectory}/FirstFoldSeekdf.csv')

In [53]:
FirstFoldSeekdf['QueryProteinLength'] = 0 # Lets initialise these columns and fill them in, this data may be useful downstream
FirstFoldSeekdf['TargetProteinLength'] = FirstFoldSeekdf['tseq'].apply(lambda x: len(x))

for query in list(FirstFoldSeekdf['query'].unique()): # we are looping through each unique query to calculate their length
    queryUnique = query.split('_')[0] # pdb files can contain multiple chains
    NumChains = sum(query.split('_')[0] in pdbfile for pdbfile in list(FirstFoldSeekdf['query'].unique())) # count chains - need to divide by this after as it will count amino acids from all chains!
    ProteinLength = calculate_protein_length(f'{QueryPath}/{queryUnique}')/NumChains
    FirstFoldSeekdf.loc[FirstFoldSeekdf['query'] == query, 'QueryProteinLength'] = ProteinLength

## Obtaining variables from BLASTs
### NOTE: There will be many repeated target sequences - infact *most of them will be* if you have many structurally similar queries - to not make the BLASTs horrifically inefficent, we will take the unique sequences and BLAST those and then join back to the main dataframe using the sequences('tseq') column

In [80]:
UniqueBLASTDF = pd.DataFrame()
UniqueBLASTDF['tseq'] = FirstFoldSeekdf['tseq'].unique()
UniqueBLASTDF

,tseq
0,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
1,MVLSDAEWQLVLNIWAKVEADVAGHGQDILIRLFKGHPETLEKFDK...
2,MVLSEGEWQLVLHVWAKVEADIAGHGQDILIRLFKHHPETLEKFDR...
3,MVLSDAEWQLVLNIWAKVEADVAGHGQDILIRLFKGHPETLEKFDK...
4,MGLSDGEWQLVLNVWGKVEADLAGHGQDILIRLFKGHPETLEKFDK...
...,...
8725,VCFAMLLLVLPLGRKRNGQVTVTQPPASVRLEGTVTISATGSSDIG...
8726,TTNLLNDTKCTGFLLWLIHTDKSKSVSLGATVTISATGSSNIGADL...
8727,MSCCYSYILDEKQTELDFVICFCTASQGVDDDLSCYLQKPGQPPKL...
8728,MTPTADSCPSSLLTQSPSLAGPVRVLAVSWYQQQQRKAPKLLIYYA...


In [83]:
# This can take just as much time as the FoldSeek itself depending on how many BLASTs you're doing and the database size!

for BLAST in BLASTList: # Loop through each BLAST we need to do and BLAST the whole 'tseq' column
    BLASTInput = glob.glob(f'BLASTDatabases/{BLAST}_BLAST_DB/*.pdb')[0].split('.pdb')[0] # this will be the .fa file for those rebuilt, or for those not rebuilt
                                                                 # from fasta e.g. patent - just the string 'pataa' which is what BLAST wants
    BLASTOutput = f'{BLASTOutputDirectory}/{BLAST}_Output.xml' # Specific output .xml file for this BLAST
    GetBestBlastAlignmentBitscoreEff(UniqueBLASTDF['tseq'], # This function BLASTs the whole column at once
                                  BLASTInput,
                                  BLASTOutputDirectory,
                                  BLASTOutput)

### I split up these loops as running BLAST requires significantly more computation than parsing, and conceptually it's helpful to split them up 

In [105]:
for BLAST in BLASTList:
    UniqueBLASTDF = add_blast(UniqueBLASTDF,f'{BLASTOutputDirectory}/{BLAST}_Output.xml',BLAST)
UniqueBLASTDF

,tseq,BPPRC Blastbitscores,BPPRC Blast raw_bistscores,BPPRC Blast eval,BPPRC Blast aln len,BPPRC Blast name,BPPRC Blast positives,BPPRC Blast identities,BPPRC Blast gaps,AllergenOnline Blastbitscores,...,Patent Blast raw_bistscores,Patent Blast eval,Patent Blast aln len,Patent Blast name,Patent Blast positives,Patent Blast identities,Patent Blast gaps,BPPRC Blast bitscores,AllergenOnline Blast bitscores,Patent Blast bitscores
0,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,0.0000,0,10.000000,0,no protein,0,0,0,35.4242,...,791,2.472320e-108,154,Sequence 10 from patent US 10934531,154,154,0,0.0000,35.4242,309.3010
1,MVLSDAEWQLVLNIWAKVEADVAGHGQDILIRLFKGHPETLEKFDK...,25.4090,54,1.329000,95,,39,23,4,35.0390,...,785,1.594730e-107,153,Sequence 2 from patent US 7164991,153,153,0,25.4090,35.0390,306.9900
2,MVLSEGEWQLVLHVWAKVEADIAGHGQDILIRLFKHHPETLEKFDR...,0.0000,0,10.000000,0,no protein,0,0,0,35.4242,...,768,6.753590e-105,154,Sequence 10 from patent US 10934531,152,149,0,0.0000,35.4242,300.4420
3,MVLSDAEWQLVLNIWAKVEADVAGHGQDILIRLFKGHPETLEKFDK...,25.4090,54,1.329000,95,,39,23,4,35.0390,...,780,1.149810e-106,153,Sequence 2 from patent US 7164991,152,152,0,25.4090,35.0390,305.0640
4,MGLSDGEWQLVLNVWGKVEADLAGHGQDILIRLFKGHPETLEKFDK...,27.3350,59,0.342556,59,,31,17,4,33.1130,...,729,7.370430e-99,154,Sequence 10 from patent US 10934531,149,140,0,27.3350,33.1130,285.4190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8725,VCFAMLLLVLPLGRKRNGQVTVTQPPASVRLEGTVTISATGSSDIG...,24.2534,51,3.031490,46,,21,16,8,25.0238,...,285,2.538960e-32,99,Sequence 5 from patent US 9758586,74,59,4,24.2534,25.0238,114.3900
8726,TTNLLNDTKCTGFLLWLIHTDKSKSVSLGATVTISATGSSNIGADL...,23.0978,48,4.796600,37,,21,13,0,22.7126,...,293,8.385620e-34,93,Sequence 12 from patent US 11008400,71,57,0,23.0978,22.7126,117.4720
8727,MSCCYSYILDEKQTELDFVICFCTASQGVDDDLSCYLQKPGQPPKL...,25.0238,53,0.979482,59,,25,17,2,24.2534,...,222,5.478150e-23,74,Sequence 8 from patent US 10266600,51,43,0,25.0238,24.2534,90.1225
8728,MTPTADSCPSSLLTQSPSLAGPVRVLAVSWYQQQQRKAPKLLIYYA...,25.7942,55,0.413379,39,,22,11,2,25.4090,...,270,2.325440e-30,79,Sequence 17034 from patent US 9254311,59,54,0,25.7942,25.4090,108.6120


In [106]:
BLASTeddf = pd.merge(FirstFoldSeekdf, UniqueBLASTDF, on='tseq', how='inner')
BLASTeddf.to_csv(f'{CSVFileDirectory}/BLASTedFoldSeekdf.csv',index=False)

### Here may again be a good place to stop and visualise what all this extra data looks like

## Filtering based on structure and BLAST
### This stage is entirely User-dependent, filter don't filter, noodle don't noodle... It's all up to you (and those parameters you set at the start)

In [107]:
StructFiltdf = BLASTeddf[(BLASTeddf['qtmscore']>=TMScoreThreshold) & (BLASTeddf['ttmscore']>=TMScoreThreshold)]
StructFiltdf.to_csv(f'{CSVFileDirectory}/StructFiltdf.csv',index=False)

In [111]:
BLASTFiltdf = StructFiltdf
for BLAST, thresh in zip(BLASTList, BLASTBitscoreThresholds): # Make sure the lists are the same length!
    BLASTFiltdf = BLASTFiltdf[BLASTFiltdf[f'{BLAST} Blast bitscores'] <= thresh]
BLASTFiltdf.to_csv(f'{CSVFileDirectory}/BLASTFiltdf.csv',index=False)

In [112]:
if MustStartWithMethionine:
    MethionineFiltdf = BLASTFiltdf[BLASTFiltdf['tseq'].str.startswith('M')]
    MethionineFiltdf.to_csv(f'{CSVFileDirectory}/MethionineFiltdf.csv',index=False)

In [124]:
if RemoveDuplicates:
    DuplicateRemoveddf = MethionineFiltdf.drop_duplicates(subset='tseq')
    DuplicateRemoveddf.to_csv(f'{CSVFileDirectory}/DuplicateRemoveddf.csv',index=False)